---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [ ]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def blight_model():
    import pandas as pd
    import numpy as np
    
    # Your code here
    train = pd.read_csv('./train.csv',encoding='latin1' )
    test = pd.read_csv('./test.csv', encoding='latin1')
    address = pd.read_csv('./addresses.csv', encoding='latin1')
    lat = pd.read_csv('./latlons.csv' ,encoding='latin1')
    
    ## Merging the train and test dataset with the latitude and address
    new_train = pd.merge(train, address, on = 'ticket_id')
    train = pd.merge(new_train, lat, on = 'address')
    
    new_test = pd.merge(test, address, on = 'ticket_id')
    test = pd.merge(new_test, lat, on = 'address')
    
    # Delete the features that are not present in Test set
    Features_drop_from_train_dataset = ['payment_amount', 'payment_date',
                                        'payment_status', 'balance_due', 
                                        'collection_status', 'compliance_detail' ]
    train.drop(Features_drop_from_train_dataset, axis = 1, inplace = True)
    
    # Remove the null part of the target
    train = train[~train.compliance.isnull()]
    
    # Remove from train set which is not  USA
    train = train[train.country == 'USA']
    
    # From careful observation, it was shown that the state which is not Michigan has almost the same address as Michigan, thus I suggest the state were all Michigan but to ensure the data is messy different state is imputed.
    # And also after careful online search of cities in MI, it was discovered that there are over 1357 cities, thus it will also be discarded.
    # The only feature which talks about location lat and lon will be used for the continuation of the prediction
    
    # Remove lat and lon outliers
    train = train[train.lat < 44.5]
    train = train[~((train.lon < -84.0) | (train.lon > -82.7))]
    
    # Features to be removed from both the train and test set
    Features_to_be_removed = ['mailing_address_str_name', 'violation_description','address', 'grafitti_status',
                              'violation_street_number', 'violation_street_name',
                         'inspector_name', 'violator_name', 'violation_zip_code',
                              'mailing_address_str_number','non_us_str_code', 'country', 
                              'ticket_issued_date', 'hearing_date', 'city', 'state', 'zip_code']
    train.drop(Features_to_be_removed, axis =1, inplace = True)
    
    # Remove the ticket_id from the train set only
    train.drop('ticket_id', axis =1 , inplace= True)
    test.drop(Features_to_be_removed, axis = 1, inplace=True)
    
    # Create a variable for the test ticket_ID
    ticket_ID = test.ticket_id

    # Drop the test ticket_id
    test.drop('ticket_id', axis =1 , inplace= True)
    
    # Dealing with categorical variables left
    test.disposition.replace('Responsible - Compl/Adj by Default', 'Responsible by Default', inplace = True)
    test.disposition.replace('Responsible - Compl/Adj by Determi', 'Responsible by Determination', inplace = True)
    test.disposition.replace('Responsible (Fine Waived) by Admis', 'Responsible (Fine Waived) by Deter', inplace= True)
    test.disposition.value_counts()
    
    violation_code_stats = train.groupby('violation_code')['violation_code'].agg('count').sort_values(ascending = False)
    violation_code_stats_less_than_50 =violation_code_stats[violation_code_stats <50]
    
    violation_code_test = test.groupby('violation_code')['violation_code'].agg('count').sort_values(ascending = False)
    violation_code_test_less_than_50 =violation_code_test[violation_code_test <50]
    
    train.violation_code = train.violation_code.apply(lambda x: 'others' if x in violation_code_stats_less_than_50 else x)
    test.violation_code = test.violation_code.apply(lambda x: 'others' if x in violation_code_test_less_than_50 else x)
    
    test.lat = test.lat.fillna(np.mean(test.lat))
    test.lon = test.lon.fillna(np.mean(test.lon))
    # Getting the categorical features for encoding
    cat_feat = train.select_dtypes(include=[np.object]).columns

    
    # import the encoder OneHotEncoder
    
    # Fitting the encoder to the categorical variables
    X = train.drop('compliance', axis = 1)
    y = train.compliance.values
    valid = test.copy()
    X = X.reset_index(drop = True)
    
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    label_encoder.fit(X['disposition'].append(valid['disposition'], ignore_index=True))
    X['disposition'] = label_encoder.transform(X['disposition'])
    valid['disposition'] = label_encoder.transform(valid['disposition'])
    label_encoder = LabelEncoder()
    label_encoder.fit(X['violation_code'].append(valid['violation_code'], ignore_index=True))
    X['violation_code'] = label_encoder.transform(X['violation_code'])
    valid['violation_code'] = label_encoder.transform(valid['violation_code'])
    label_encoder = LabelEncoder()
    label_encoder.fit(X['agency_name'].append(valid['agency_name'], ignore_index=True))
    X['agency_name'] = label_encoder.transform(X['agency_name'])
    valid['agency_name'] = label_encoder.transform(valid['agency_name'])
    
    from sklearn.preprocessing import OneHotEncoder
    ohe = OneHotEncoder(handle_unknown='ignore' ) 
    # categorical_features = boolean mask for categorical columns
    # sparse = False output an array not sparse matrix
    train_cat = ohe.fit_transform(X.iloc[:,[0,1,2]]).toarray() # It returns an numpy array
    test_cat = ohe.transform(valid.iloc[:,[0,1,2]]).toarray()
   
    train_cat = pd.DataFrame(train_cat)
    test_cat = pd.DataFrame(test_cat)
    
    X = X.join(train_cat)
    valid = valid.join(test_cat)
    
    # Drop the categorical variable after encoding
    X.drop(cat_feat, axis = 1, inplace= True)
    valid.drop(cat_feat, axis = 1, inplace= True)
    
    # Import the algorithm and the metrics score
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
    
    gbt_clf = GradientBoostingClassifier(random_state= 422)
    gbt_clf.fit(X, y)
    
    df = pd.Series(gbt_clf.predict_proba(valid)[:,1], ticket_ID)
    return df
blight_model()

In [ ]:

Your AUC of 0.809528864096 was awarded a value of 1.0 out of 1.0 total grades

In [ ]:
blight_model()